# Мини-проект #1:

# Описание данных:
Имеются следующие переменные:  
  
- Hotel – тип отеля (City Hotel или Resort Hotel)  
- Is canceled – бронирование было отменено (1) или нет (0); неотменённое считается успешным
- Lead time – количество дней, прошедших между датой бронирования и датой прибытия  
- Arrival full date – полная дата прибытия
- Arrival date year – год прибытия  
- Arrival date month – месяц прибытия  
- Arrival date week number – номер недели прибытия
- Arrival date day of month – день прибытия
- Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость забронировал для проживания в отеле
- Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал для проживания в отеле
- Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)
- Adults – число взрослых
- Children – число детей
- Babies – число младенцев 
- Meal – выбранный тип питания
- Country – страна происхождения клиента
- Reserved room type – тип зарезервированного номера
- Assigned room type – тип полученного номера (может отличаться от забронированного)
- Customer type – тип бронирования
- Reservation status – значение последнего статуса брони: Canceled – было отменено клиентом; Check-Out – клиент зарегистрировался, но уже покинул отель; No-Show – клиент не зарегистрировался и сообщил администрации отеля причину
- Reservation status date – дата обновления статуса

### Будем анализировать данные о бронировании отелей.
<b>В качестве результатов получим данные по следующим вопросам:</b>  
- В каких странах совершено наибольшее число успешных бронирований;    
- На сколько ночей в среднем бронируют отели разных типов;    
- Сколько раз номер фактический номер клиента отличается от забронированного (Такое может произойти, например, по причине овербукинга);  
- На какой месяц чаще всего успешно оформляли бронь в 2016 и изменился ли самый популярный месяц в 2017;  
- Проверим, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов по годам;  
- Определим среднее значение характеристик adults, children и babies;  
- Определим, отели какого типа в среднем пользуются большей популярностью у клиентов с детьми;    
- Посчитаем churn rate у разных групп клиентов;    

# 1. Импорт данных и построение DataFrame

In [3]:
import pandas as pd
bookings = pd.read_csv('https://stepik.org/media/attachments/lesson/360344/bookings.csv',sep=';')

In [7]:
bookings.head()

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


In [6]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Hotel                      119390 non-null  object 
 1   Is Canceled                119390 non-null  int64  
 2   Lead Time                  119390 non-null  int64  
 3   arrival full date          119390 non-null  object 
 4   Arrival Date Year          119390 non-null  int64  
 5   Arrival Date Month         119390 non-null  object 
 6   Arrival Date Week Number   119390 non-null  int64  
 7   Arrival Date Day of Month  119390 non-null  int64  
 8   Stays in Weekend nights    119390 non-null  int64  
 9   Stays in week nights       119390 non-null  int64  
 10  stays total nights         119390 non-null  int64  
 11  Adults                     119390 non-null  int64  
 12  Children                   119386 non-null  float64
 13  Babies                     11

Предобработка данных: заменим названия столбоц на строчные буквы, а пробелы на нижний слэш:

In [310]:
bookings.columns = bookings.columns.str.replace(' ','_').str.lower()

Удалим дубликатыв и пустые значения в данных:

In [278]:
bookings = bookings.drop_duplicates().reset_index(drop = True)

Для исследования нам важны столбцы: 
- hotel, is_canceled; 
- arrival_date_year;  
- arrival_date_month; 
- adults; 
- children; 
- babies; 
- country; 
- reserved_room_type; 
- assigned_room_type;  
Значит, мы должны удалить из таблицы пустые значения, если они встречаются в приведённых столбцах таблицы. Если они есть в других стобцах, но отсутствуют в нужных нам, то мы оставим строки в таблице

In [279]:
bookings.dropna(subset=['hotel', 'is_canceled', 'arrival_date_year', 'arrival_date_month', 'adults', 'children', 'babies', 'country', 'reserved_room_type', 'assigned_room_type'], inplace = True)

In [280]:
bookings

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83150,City Hotel,0,23,2017-08-30,2017,August,35,30,2,5,...,2,0.0,0,BB,BEL,A,A,Transient,Check-Out,2017-09-06
83151,City Hotel,0,102,2017-08-31,2017,August,35,31,2,5,...,3,0.0,0,BB,FRA,E,E,Transient,Check-Out,2017-09-07
83152,City Hotel,0,34,2017-08-31,2017,August,35,31,2,5,...,2,0.0,0,BB,DEU,D,D,Transient,Check-Out,2017-09-07
83153,City Hotel,0,109,2017-08-31,2017,August,35,31,2,5,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2017-09-07


Мы закончили предобработку данных, теперь можно приступать к анализу

# 2. Определим топ 5 стран, пользователи которых совершили наибольшее число успешных бронирований?

Успешное бронирование - то, где показатель 'is_canceled' == 0, что означает, что бронирование не отменилось

In [281]:
bookings[bookings['is_canceled']==0].groupby('country') \
    .count() \
    .sort_values('hotel',ascending=False) \
    .head(5)['hotel']

country
PRT    16138
GBR     7803
FRA     6808
ESP     5076
DEU     4130
Name: hotel, dtype: int64

<b>Вывод:</b> Чаще всего успешные бронирования отелей совершают граждане следующих стран:  
    1) Португалия  
    2) Великобритания  
    3) Франция  
    4) Испания  
    5) Германия  

Проверим, отличется ли данный список от Топ-5 стран по общему количеству бронирований:

In [282]:
bookings.groupby('country') \
    .count() \
    .sort_values('hotel',ascending=False) \
    .head(5)['hotel']

country
PRT    25720
GBR     9770
FRA     8526
ESP     6900
DEU     5169
Name: hotel, dtype: int64

<b>Вывод:</b> Как видно, топ-5 стран не изменился, однако можно отметить, что по количеству отменённых бронирований Португалия также является лидером среди этих пяти стран.

# 3. На сколько ночей в среднем бронируют отели разных типов?

Количество ночей, на которые бронируется номер отражены в колонке 'stays_total_nights'

In [283]:
bookings.groupby('hotel') \
    .agg({'stays_total_nights':'mean'}) \
    .round(2)

,stays_total_nights
hotel,
City Hotel,3.17
Resort Hotel,4.46


<b>Вывод:</b> Среднее число забронированных ночей в городских отелях - 3,17; в курортных - 4,46

# 4. Выявим количество несоответствий забронированых номеров полученным?

Для получения данных мы будем пользоваться колонками 'reserved_room_type' и 'assigned_room_type'

In [284]:
bookings[bookings['reserved_room_type'] != bookings['assigned_room_type']].shape[0]

12116

Также было бы неплохо посчитать отношение таких номеров к общему количеству:

In [285]:
ratio = bookings[bookings['reserved_room_type'] != bookings['assigned_room_type']].shape[0] / bookings.shape[0]
(round(ratio,2) * 100)

15.0

<b>Вывод:</b> Несоответствие заявленного номера фактическому происходит в 15% случаев

# 5. Определим, на какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?

In [286]:
bookings[(bookings['arrival_date_year']==2016) | (bookings['arrival_date_year']==2017)].groupby(['arrival_date_year','arrival_date_month']) \
    .count() \
    .sort_values(by='hotel',ascending=False)

hotel  is_canceled  lead_time  \
arrival_date_year arrival_date_month                                  
2017              July                 4403         4403       4403   
                  May                  4368         4368       4368   
2016              August               4270         4270       4270   
2017              August               4269         4269       4269   
                  June                 4115         4115       4115   
2016              October              3935         3935       3935   
2017              April                3925         3925       3925   
2016              July                 3689         3689       3689   
                  September            3602         3602       3602   
                  March                3583         3583       3583   
                  April                3558         3558       3558   
                  May                  3557         3557       3557   
2017              March                3478         3478       3478   
2016              June                 3346         3346       3346   
                  November             3158         3158       3158   
2017              February             3131         3131       3131   
2016              December             2970         2970       2970   
2017              January              2665         2665       2665   
2016              February             2580         2580       2580   
                  January              1670         1670       1670   

                                      arrival_full_date  \
arrival_date_year arrival_date_month                      
2017              July                             4403   
                  May                              4368   
2016              August                           4270   
2017              August                           4269   
                  June                             4115   
2016              October                          3935   
2017              April                            3925   
2016              July                             3689   
                  September                        3602   
                  March                            3583   
                  April                            3558   
                  May                              3557   
2017              March                            3478   
2016              June                             3346   
                  November                         3158   
2017              February                         3131   
2016              December                         2970   
2017              January                          2665   
2016              February                         2580   
                  January                          1670   

                                      arrival_date_week_number  \
arrival_date_year arrival_date_month                             
2017              July                                    4403   
                  May                                     4368   
2016              August                                  4270   
2017              August                                  4269   
                  June                                    4115   
2016              October                                 3935   
2017              April                                   3925   
2016              July                                    3689   
                  September                               3602   
                  March                                   3583   
                  April                                   3558   
                  May                                     3557   
2017              March                                   3478   
2016              June                                    3346   
                  November                                3158   
2017              February                            

<b>Вывод:</b> самым популярным месяцем для бронирования отелей в 2016 году был Август, тогда как в 2017 таким месяцем стал Июль.

# 6. Проверим, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов по годам.

In [287]:
bookings[(bookings['hotel']=='City Hotel') | (bookings['is_canceled']==1)].groupby(['arrival_date_year','arrival_date_month']) \
    .count() \
    .sort_values('is_canceled',ascending=False)

hotel  is_canceled  lead_time  \
arrival_date_year arrival_date_month                                  
2017              July                 3390         3390       3390   
                  May                  3355         3355       3355   
2016              August               3250         3250       3250   
2017              August               3238         3238       3238   
                  June                 3181         3181       3181   
                  April                2971         2971       2971   
2016              October              2894         2894       2894   
                  July                 2804         2804       2804   
                  September            2784         2784       2784   
2017              March                2636         2636       2636   
2016              April                2590         2590       2590   
                  May                  2538         2538       2538   
                  March                2498         2498       2498   
                  June                 2478         2478       2478   
                  November             2279         2279       2279   
2017              February             2182         2182       2182   
2016              December             2158         2158       2158   
2017              January              1826         1826       1826   
2015              September            1816         1816       1816   
2016              February             1717         1717       1717   
2015              October              1633         1633       1633   
                  August               1373         1373       1373   
                  December             1102         1102       1102   
2016              January              1048         1048       1048   
2015              November              857          857        857   
                  July                  636          636        636   

                                      arrival_full_date  \
arrival_date_year arrival_date_month                      
2017              July                             3390   
                  May                              3355   
2016              August                           3250   
2017              August                           3238   
                  June                             3181   
                  April                            2971   
2016              October                          2894   
                  July                             2804   
                  September                        2784   
2017              March                            2636   
2016              April                            2590   
                  May                              2538   
                  March                            2498   
                  June                             2478   
                  November                         2279   
2017              February                         2182   
2016              December                         2158   
2017              January                          1826   
2015              September                        1816   
2016              February                         1717   
2015              October                          1633   
                  August                           1373   
                  December                         1102   
2016              January                          1048   
2015              November                          857   
                  July                              636   

                                      arrival_date_week_number  \
arrival_date_year arrival_date_month                             
2017              July                                    3390   
                  May                                     3355   
2016              August                                  3250   
2017              August                                  3238   


<b>Вывод:</b> В 2017 бронь чаще всего отменялась в Июле, в 2016 - в Августе. Это может быть вызвано тем, что на эти месяцы в принципе приходилось наибольшее количество бронирований

# 7. Определим среднее значение характеристик adults, children и babies.

In [311]:
bookings.agg({'adults':'mean','children':'mean','babies':'mean'}).round(2)

adults      1.86
children    0.10
babies      0.01
dtype: float64

<b>Вывод:</b> В среднем люди едут отдыхать вдвоём. Очень редко (в 10% случаев берут с собой детей), и практически не берут младенцев

# 8. Определим, отели какого типа в среднем пользуются большей популярностью у клиентов с детьми?

Добавим колонку, показывающую, имеются ли дети у забронировавшего человека:

In [324]:
bookings['have_children'] = ((bookings['children'] > 0) | (bookings['babies'] > 0))
bookings['total_kids'] = bookings['children'] + bookings['babies']

Оставим только клиентов, имеющих детей

In [325]:
bookings.groupby('hotel') \
    .agg({'total_kids':'mean'}) \
    .round(2)

,total_kids
hotel,
City Hotel,0.10
Resort Hotel,0.14


<b>Вывод:</b> Клиенты с детьми чаще предпочитают отели типа Resort

# 9. Посчитаем churn rate у разных групп клиентов.

Churn rate (отток, коэффициент оттока) – это процент подписчиков (например, на push-уведомления от сайта), которые отписались от канала коммуникации, отказались от услуг сервиса в течение определенного периода времени. Иными словами, представляет собой отношение количества ушедших пользователей к общему количеству пользователей, выраженное в процентах.

В данном задании сопоставим средние значения у churn rate для клиентов с детьми и для клиентов без детей

In [332]:
ave_child = bookings[(bookings['is_canceled']==1) & (bookings['have_children']==True)].shape[0] / bookings[bookings['have_children']==True].shape[0] * 100
round(ave_child,2)

34.92

In [333]:
ave = bookings[(bookings['is_canceled']==1) & (bookings['have_children']==False)].shape[0] / bookings[bookings['have_children']==False].shape[0] * 100
round(ave,2)

37.22

In [334]:
ave_final = bookings[bookings['is_canceled']==1].shape[0] / bookings.shape[0] * 100
round(ave_final,2)

37.04

In [335]:
print('Средний Chutn Rate для всех клиентов составляет {}%.\nChutn Rate у клиентов с детьми составляет {}%\nСhutn Rate у клиентов без детей составляет {}%'.format(round(ave_final,2),round(ave_child,2),round(ave,2)))


Средний Chutn Rate для всех клиентов составляет 37.04%.
Chutn Rate у клиентов с детьми составляет 34.92%
Сhutn Rate у клиентов без детей составляет 37.22%


<b>Вывод:</b> в среднем, клиенты с детьми отменяют заказы чаще, чем без детей, но не сильно (Δ ~ 3-4%)